# Este codigo prepara los datos para usarlos con pytorch (generando un archivo CSV)

In [14]:
# Imports
import pandas as pd
import numpy as np

In [15]:
# consts
cats = ["MEL","NV","BCC","AK","BKL","DF","VASC","SCC","UNK"]

## Sacamos los datos del CSV

In [16]:
csv = pd.read_csv("./dataset/ISIC_2019_Training_GroundTruth.csv")

print("> Cantidad de elementos", csv.count(axis=1).size )
print("> Head de los datos del csv")
print(csv.head())

> Cantidad de elementos 25331
> Head de los datos del csv
          image  MEL   NV  BCC   AK  BKL   DF  VASC  SCC  UNK
0  ISIC_0000000  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
1  ISIC_0000001  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
2  ISIC_0000002  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
3  ISIC_0000003  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
4  ISIC_0000004  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0


## Sacamos las categorias (esto no es necesario)

In [17]:
header = list(csv.columns)
header.remove("image")

print("> Categorias")
print(header)

> Categorias
['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']


## Creamos un Dataframe en el formato que requiere pytorch (imagen<string>, categoria<int>)

In [18]:
data = pd.DataFrame({"img":[], "cat": []}, dtype=int)

for entry in csv.values:
    data = data.append({"img":entry[0] , "cat":np.where(entry==1.0)[0][0]-1}, ignore_index=True)


print("> Cantidad de filas", csv.count(axis=1).size)
print("> Head de los datos en el formato que usa pytorch")
print(data.head())

> Cantidad de filas 25331
> Head de los datos en el formato que usa pytorch
            img  cat
0  ISIC_0000000    1
1  ISIC_0000001    1
2  ISIC_0000002    0
3  ISIC_0000003    1
4  ISIC_0000004    0


## Contamos cuantas imagenes hay de cada categoria

In [19]:
counter = np.zeros(len(cats), dtype=int)

for elm in data.values:
    counter[int(elm[1])]+=1

print("> Cantidad de imagenes de cada tipo:")
print(counter)

> Cantidad de imagenes de cada tipo:
[ 4522 12875  3323   867  2624   239   253   628     0]


## Averiguamos cual es el tamaño del grupo de imagenes mas pequeño

In [20]:
# counter = list(filter(lambda elm: elm > 0, counter))
# min_cat_size = min(counter)

min_cat_size = data.cat.value_counts().min()

print("> Cantidad mínima de imagenes de un tipo:")
print(min_cat_size)

> Cantidad mínima de imagenes de un tipo:
239


## Hacemos que todos los grupos tengan el mismo tamaño, eligiendolos de forma aleatoria

In [21]:
data = data.groupby("cat").apply(lambda cat: cat.sample(min_cat_size)).reset_index(drop=True)

print("> Cantidad de elemntos:", data.count(axis=1).size )
print("> Datos equilibrados")
print(data.head())

> Cantidad de elemntos: 1912
> Datos equilibrados
            img  cat
0  ISIC_0055317    0
1  ISIC_0056663    0
2  ISIC_0054251    0
3  ISIC_0064463    0
4  ISIC_0056479    0


## Barajamos todas las filas del DataFrame

In [22]:
data = data.sample(frac=1)

print("> cantidad de elementos", data.count(axis=1).size)
print("> datos barajados")
print(data.head())

> cantidad de elementos 1912
> datos barajados
               img  cat
1576  ISIC_0067285    6
305   ISIC_0031663    1
329   ISIC_0067172    1
1316  ISIC_0033810    5
559   ISIC_0029680    2


## Comprobamos cuantas imagenes hay de cada categoria

In [23]:
counter = np.zeros(len(cats), dtype=int)

for elm in data.values:
    counter[int(elm[1])]+=1

print("> Cantidad de imagenes de cada tipo:")
print(counter)

> Cantidad de imagenes de cada tipo:
[239 239 239 239 239 239 239 239   0]


## Guardamos el resultado en un archivo CSV
> Esto para mas eficiencia a la hora de usarlo, mantener siempre el mismo dataset y evitar la sobrerrepresentación de las categorias con mas elementos.

In [24]:
data.to_csv("./dataset/balanced_data.csv", index=False)